In [ ]:
import sqlalchemy as sql
import psycopg2 as pc
import pandas as pd
import numpy as np
import spacy as sp
from neo4j import GraphDatabase
from neo4j.exceptions import ServiceUnavailable
import csv
import json
from jsonpath_ng.ext import parse 

In [ ]:
awards_filepath = "src/files/220k_awards_by_directors.csv"
movies_filepath     = "src/files/AllMoviesDetailsCleaned.csv"
cast_filepath       = "src/files/AllMoviesCastingRaw.csv"

awards_df = pd.read_csv(awards_filepath, delimiter=",")
movies_df = pd.read_csv(movies_filepath, delimiter=";")
cast_df = pd.read_csv(cast_filepath, delimiter=";")

In [ ]:
def processMovies(df, movie_file):
    movie_header = ["title:ID", ]
    company_header = 
    movie_data{}

In [ ]:


def processAwards(node_file, relation_file):
    node_header = ["patentId:ID", "assignee:string[]", ":LABEL"]
    relation_header = [":START_ID",":TYPE", ":END_ID"]
    node_data = {}
    relation_data = []
    # get nodes
    for i, row in df.iterrows():
        patentid = row["patentid"]
        citations = row["cite"]
        
        # Skip if either are empty
        if patentid is None or citations is None:
            continue;

        # Ensure all nodes are included. Use cite data for assignee
        if (patentid not in node_data):
            node_data[patentid] = [patentid, "", "Patent"]
            
        # Get backward references (node CITES other)
        if ("backwardReferences" in citations):
            for backRef in citations["backwardReferences"]:
                cite_id = backRef[0]
                cite_date = backRef[1]
                cite_assignee = list([backRef[2]])
                cite_desc = backRef[3]
                
                relation_data.append([patentid, "cites", cite_id])

                # Fill in new node
                if (cite_id not in node_data):
                    node_data[cite_id] = [cite_id, ";".join(cite_assignee), "Patent"]
                # Add assignee to existing node if empty
                elif (node_data[cite_id][1] == ""):
                    node_data[cite_id][1] = ";".join(cite_assignee)
        # Get forward references (other CITES node)
        if ("forwardReferencesFamily" in citations):
            for frontRef in citations["forwardReferencesFamily"]:
                cite_id = frontRef[0]
                cite_date = frontRef[1]
                cite_assignee = list([frontRef[2]])
                cite_desc = frontRef[3]

                relation_data.append([cite_id, "family-cites", patentid])

                if (cite_id not in node_data):
                    node_data[cite_id] = [cite_id, ";".join(cite_assignee), "Patent"]
                elif (node_data[cite_id][1] == ""):
                    node_data[cite_id][1] = ";".join(cite_assignee)
    # write nodes file 
    with open(node_file, 'w',  newline='', encoding='utf-8') as f:
        writer = csv.writer(f)
        writer.writerow(node_header)
        for row in node_data.values():
            writer.writerow(row)
    # wirte relation file 
    with open(relation_file, 'w',  newline='', encoding='utf-8') as f:
        writer = csv.writer(f)
        writer.writerow(relation_header)
        writer.writerows(relation_data) 